In [1]:
using StanModels
using CmdStan, StanMCMCChain

ProjDir = rel_path_s("..", "scripts", "12")

d = CSV.read(rel_path( "..", "data",  "Kline.csv"), delim=';');
size(d) # Should be 10x5

(10, 5)

New col log_pop, set log() for population data

In [2]:
d[:log_pop] = map((x) -> log(x), d[:population]);
d[:society] = 1:10;

first(d, 5)

m12_6_2 = "
  data {
    int N;
    int T[N];
    int N_societies;
    int society[N];
    int P[N];
  }
  parameters {
    real alpha;
    vector[N_societies] a_society;
    real bp;
    real<lower=0> sigma_society;
  }
  model {
    vector[N] mu;
    target += normal_lpdf(alpha | 0, 10);
    target += normal_lpdf(bp | 0, 1);
    target += cauchy_lpdf(sigma_society | 0, 1);
    target += normal_lpdf(a_society | 0, sigma_society);
    for(i in 1:N) mu[i] = alpha + a_society[society[i]] + bp * log(P[i]);
    target += poisson_log_lpmf(T | mu);
  }
  generated quantities {
    vector[N] log_lik;
    {
    vector[N] mu;
    for(i in 1:N) {
      mu[i] = alpha + a_society[society[i]] + bp * log(P[i]);
      log_lik[i] = poisson_log_lpmf(T[i] | mu[i]);
    }
    }
  }
";

Define the Stanmodel and set the output format to :mcmcchain.

In [3]:
stanmodel = Stanmodel(name="m12.6.2",  model=m12_6_2,
output_format=:mcmcchain);


File /Users/rob/.julia/dev/StanModels/notebooks/12/tmp/m12.6.2.stan will be updated.



Input data for cmdstan

In [4]:
m12_6_2_data = Dict("N" => size(d, 1), "T" => d[:total_tools],
"N_societies" => 10, "society" => d[:society], "P" => d[:population]);

Sample using cmdstan

In [5]:
rc, chn, cnames = stan(stanmodel, m12_6_2_data, ProjDir,
diagnostics=false, summary=false, CmdStanDir=CMDSTAN_HOME);

Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
Exception: normal_lpdf: Scale parameter is 0, but must be > 0!  (in '/Users/rob/.julia/dev/StanModels/notebooks/12/tmp/m12.6.2.stan' at line 19)

If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.



Describe the draws

In [6]:
describe(chn)

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
                   Mean           SD         Naive SE        MCSE         ESS    
         lp__ -39.619071600  3.0241928411 0.047816687307 0.0751213413 1000.000000
accept_stat__   0.941207732  0.0815449287 0.001289338531 0.0015989632 1000.000000
   stepsize__   0.045981300  0.0027642555 0.000043706717 0.0004425798   39.009752
  treedepth__   5.946250000  0.4334834651 0.006853975388 0.0148310464  854.282457
 n_leapfrog__  77.042000000 34.3888046240 0.543734743113 1.0175404736 1000.000000
  divergent__   0.000000000  0.0000000000 0.000000000000 0.0000000000         NaN
     energy__  46.125880875  3.8994470277 0.061655671114 0.0957828046 1000.000000
        alpha   1.105122592  0.7466474328 0.011805532483 0.0166971260 1000.000000
  a_society.1  -0.205773072  0.2439030304 0.003856445522 0.0057541985 1000.000000
  a_society.2   0.047975160  0.2187265158 0.003458369872 0.00447

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*